In [9]:
import gym
import overcookedgym
from stable_baselines3 import PPO
from pantheonrl.common.agents import OnPolicyAgent
from overcookedgym.overcooked_utils import LAYOUT_LIST
import os
import numpy as np
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from pantheonrl.common.agents import StaticPolicyAgent
import matplotlib.pyplot as plt

c:\Users\bpcur\anaconda3\envs\PantheonRL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LAYOUT_LIST = ['corridor', 'five_by_five', 'mdp_test', 'multiplayer_schelling',
               'random0', 'random1', 'random2', 'random3', 'scenario1_s',
               'scenario2', 'scenario2_s', 'scenario3', 'scenario4',
               'schelling', 'schelling_s', 'simple', 'simple_single',
               'simple_tomato', 'small_corridor', 'unident', 'unident_s']

NAME_TRANSLATION = {
    "cramped_room": "simple",
    "asymmetric_advantages": "unident_s",
    "coordination_ring": "random1",
    "forced_coordination": "random0",
    "counter_circuit": "random3",
}

# Functions

In [10]:
def reset_env(layout):
    environment_name = 'OvercookedMultiEnv-v0'
    assert layout in LAYOUT_LIST
    env = gym.make(environment_name , layout_name=layout)
    mon_env = Monitor(env)
    return mon_env

In [11]:
def find_checkpoints(PPO_num, cp_array, numcps):
    '''
    returns a dict of the form {PPO_num: {cp2:a, cp3:b,...}}
    this does not include cp1, as it is simply checkpoint 1

    numcps includes first and last, so numcps=3 for example will return a dict with cp2 and cp3, cp2 having the reward closest to half of cp3
    '''
    cp_array = np.append(cp_array, np.zeros((100, numcps)), axis=1)
    out = {'PPO_' + str(PPO_num): {}}
    max_val = np.max(cp_array[:, 1])
    cpvals = {}
    for i in range(1, numcps-1):
        cpvals['cp' + str(i)] = max_val*i/(numcps-1)
    for index, reward in enumerate(cp_array[:, 1]):
        for i, val in enumerate(cpvals):
            cp_array[index, i+2] = abs(cpvals[val]-reward)
        if reward == max_val:
            out['PPO_' + str(PPO_num)]['cp' + str(numcps)] = index+1
    for i in range(2, numcps):
        for index, diff in enumerate(cp_array[:, i]):
            if diff == np.min(cp_array[:, i]):
                out['PPO_' + str(PPO_num)]['cp' + str(i)] = index+1
    return out

In [12]:
def run_env(ego_path, partner_path, layout, num_episodes=10):
    '''
    runs env with specified policies for num_episodes episodes
    returns mean episode reward
    '''
    mon_env = reset_env(layout)
    ego = StaticPolicyAgent(PPO.load(ego_path))
    partner = StaticPolicyAgent(PPO.load(partner_path))
    mon_env.add_partner_agent(partner)
    scores = []

    for ep in range(num_episodes):
        obs = mon_env.reset()
        ego_obs = obs
        partner_obs = obs
        score = 0

        for t in range(400):
            ego_action = ego.policy.predict(ego_obs)
            partner_action = partner.policy.predict(partner_obs)
            (ego_obs, partner_obs), (reward1, reward2), done, info = mon_env.multi_step(ego_action[0], partner_action[0])
            score += reward1
        scores.append(score)
    
    return np.mean(scores)

In [13]:
def train_PPO_partner(num_agents, starting_num, layout):
    for m in range(starting_num, num_agents+starting_num):
        mon_env = reset_env(layout)
        partner = OnPolicyAgent(PPO('MlpPolicy', mon_env, verbose=0))
        mon_env.add_partner_agent(partner)
        mon_env.reset()
        PPO_ego = PPO('MlpPolicy', mon_env, verbose=0)
        if m<17:
            partner_save_path2 = os.path.join('saved_models', 'PPO', layout, '16m6cp', 'pair' + str(m), 'partner', 'checkpoint_1')
            ego_save_path2 = os.path.join('saved_models', 'PPO', layout, '16m6cp', 'pair' + str(m), 'ego', 'checkpoint_1')
            PPO_ego.save(ego_save_path2)
            partner.model.save(partner_save_path2)
        if m<33:
            partner_save_path = os.path.join('saved_models', 'PPO', layout, '32m3cp', 'pair' + str(m), 'partner', 'checkpoint_1')
            ego_save_path = os.path.join('saved_models', 'PPO', layout, '32m3cp', 'pair' + str(m), 'ego', 'checkpoint_1')
            PPO_ego.save(ego_save_path)
            partner.model.save(partner_save_path)
        if m>32:
            partner_save_path3 = os.path.join('saved_models', 'PPO', layout, 'eval', 'pair' + str(m-32), 'partner', 'checkpoint_1')
            ego_save_path3 = os.path.join('saved_models', 'PPO', layout, 'eval', 'pair' + str(m-32), 'ego', 'checkpoint_1')
            PPO_ego.save(ego_save_path3)
            partner.model.save(partner_save_path3)
        PPO_ego.save(ego_save_path)
        partner.model.save(partner_save_path)
        print('saved checkpoint 1 of agent ' + str(m))
        for cp in range(2, 101):
            PPO_ego.learn(total_timesteps=3000)
            if m<33:
                partner_save_path = os.path.join('saved_models', 'PPO', layout, '32m3cp', 'pair' + str(m), 'partner', 'checkpoint_' + str(cp))
                ego_save_path = os.path.join('saved_models', 'PPO', layout, '32m3cp', 'pair' + str(m), 'ego', 'checkpoint_' + str(cp))
                PPO_ego.save(ego_save_path)
                partner.model.save(partner_save_path)
            if m<17:
                partner_save_path2 = os.path.join('saved_models', 'PPO', layout, '16m6cp', 'pair' + str(m), 'partner', 'checkpoint_' + str(cp))
                ego_save_path2 = os.path.join('saved_models', 'PPO', layout, '16m6cp', 'pair' + str(m), 'ego', 'checkpoint_' + str(cp))
                PPO_ego.save(ego_save_path2)
                partner.model.save(partner_save_path2)
            if m>32:
                partner_save_path3 = os.path.join('saved_models', 'PPO', layout, 'eval', 'pair' + str(m-32), 'partner', 'checkpoint_' + str(cp))
                ego_save_path3 = os.path.join('saved_models', 'PPO', layout, 'eval', 'pair' + str(m-32), 'ego', 'checkpoint_' + str(cp))
                PPO_ego.save(ego_save_path3)
                partner.model.save(partner_save_path3)
            PPO_ego.save(ego_save_path)
            partner.model.save(partner_save_path)
            print(f"saved checkpoint {cp} of agent {m}")
        del PPO_ego

In [14]:
def evaluate_32_partners(numPPO, layout, plot=False):
    if plot:
        y = []
        x = [n for n in range(1, 101)]
    cp_array = np.array([np.arange(1, 101)]).reshape(100, 1)
    cp_array = np.append(cp_array, np.zeros((100, 1)), axis=1)
    for cp in range(1, 101):
        partner_save_path = os.path.join('saved_models', 'PPO', layout, '32m3cp', 'pair' + str(numPPO), 'partner', 'checkpoint_' + str(cp))
        ego_save_path = os.path.join('saved_models', 'PPO', layout, '32m3cp', 'pair' + str(numPPO), 'ego', 'checkpoint_' + str(cp))
        mean_reward = run_env(ego_save_path, partner_save_path, layout, 10)
        if plot:
            y.append(mean_reward)
        cp_array[cp-1, 1] = mean_reward
        print('Team ' + str(numPPO) + ', evaluated checkpoint ' + str(cp) + '. mean reward: ' + str(mean_reward))
    if plot:
        return cp_array, x, y
    return cp_array

In [15]:
def evaluate_16_partners(numPPO, layout, plot=False):
    if plot:
        y = []
        x = [n for n in range(1, 101)]
    cp_array = np.array([np.arange(1, 101)]).reshape(100, 1)
    cp_array = np.append(cp_array, np.zeros((100, 1)), axis=1)
    for cp in range(1, 101):
        partner_save_path = os.path.join('saved_models', 'PPO', layout, '16m6cp', 'pair' + str(numPPO), 'partner', 'checkpoint_' + str(cp))
        ego_save_path = os.path.join('saved_models', 'PPO', layout, '16m6cp', 'pair' + str(numPPO), 'ego', 'checkpoint_' + str(cp))
        mean_reward = run_env(ego_save_path, partner_save_path, layout, 10)
        if plot:
            y.append(mean_reward)
        cp_array[cp-1, 1] = mean_reward
        print('Team ' + str(numPPO) + ', evaluated checkpoint ' + str(cp) + '. mean reward: ' + str(mean_reward))
    if plot:
        return cp_array, x, y
    return cp_array

In [16]:
def evaluate_eval_partners(numPPO, layout, plot=False):
    if plot:
        y = []
        x = [n for n in range(1, 101)]
    cp_array = np.array([np.arange(1, 101)]).reshape(100, 1)
    cp_array = np.append(cp_array, np.zeros((100, 1)), axis=1)
    for cp in range(1, 101):
        partner_save_path = os.path.join('saved_models', 'PPO', layout, 'eval', 'pair' + str(numPPO), 'partner', 'checkpoint_' + str(cp))
        ego_save_path = os.path.join('saved_models', 'PPO', layout, 'eval', 'pair' + str(numPPO), 'ego', 'checkpoint_' + str(cp))
        mean_reward = run_env(ego_save_path, partner_save_path, layout, 10)
        if plot:
            y.append(mean_reward)
        cp_array[cp-1, 1] = mean_reward
        print('Team ' + str(numPPO) + ', evaluated checkpoint ' + str(cp) + '. mean reward: ' + str(mean_reward))
    if plot:
        return cp_array, x, y
    return cp_array

In [17]:
def add_PPO_partners(layout, checkpoints, FCPtype):
    mon_env = reset_env(layout)
    if FCPtype=='16m6cp':
        num_partners = 16
    else:
        num_partners = 32
    for ppo_num in range(1, num_partners+1):
        partner = StaticPolicyAgent(PPO.load(os.path.join('saved_models', 'PPO', layout, FCPtype, 'pair' + str(ppo_num), 'partner', 'checkpoint_1'), mon_env).policy)
        mon_env.add_partner_agent(partner)
        for cp in checkpoints['PPO_' + str(ppo_num)]:
            checkpoint = checkpoints['PPO_1'][cp]
            partner = StaticPolicyAgent(PPO.load(os.path.join('saved_models', 'PPO', layout, FCPtype, 'pair' + str(ppo_num), 'partner', 'checkpoint_' + str(checkpoint))).policy)
            mon_env.add_partner_agent(partner)
    return mon_env

In [18]:
def do_all_for_layout(layout):
    '''
    trains and evaluates both FCP models for given layout
    returns dicts of results, means, and plots in format (results32, means32, results16, means16, plots)
    '''
    train_PPO_partner(42, 1, layout)
    
    plots = {}

    keeping = {}
    numcps = 3

    x_plots_32 = []
    y_plots_32 = []

    for ppo in range(1, 33):
        partner_cp_array, x, y = evaluate_32_partners(ppo, layout, True)
        keeping.update(find_checkpoints(ppo, partner_cp_array, numcps))
        x_plots_32.append(x)
        y_plots_32.append(y)

    keeping_32 = keeping
    plots['32m3cp_partners'] = (x_plots_32, y_plots_32)

    keeping = {}
    numcps = 6

    x_plots_16 = []
    y_plots_16 = []

    for ppo in range(1, 17):
        partner_cp_array, x, y = evaluate_16_partners(ppo, layout, True)
        keeping.update(find_checkpoints(ppo, partner_cp_array, numcps))
        x_plots_16.append(x)
        y_plots_16.append(y)

    keeping_16 = keeping
    plots['16m6cp_partners'] = (x_plots_16, y_plots_16)

    keeping = {}
    numcps = 5

    x_plots_evals = []
    y_plots_evals = []

    for ppo in range(1, 11):
        partner_cp_array, x, y = evaluate_eval_partners(ppo, layout, True)
        keeping.update(find_checkpoints(ppo, partner_cp_array, numcps))
        x_plots_evals.append(x)
        y_plots_evals.append(y)

    keeping_evals = keeping
    plots['eval_partners'] = (x_plots_evals, y_plots_evals)

    FCPtype = '32m3cp'
    mon_env = reset_env(layout)
    mon_env = add_PPO_partners(layout, keeping_32, FCPtype)
    FCP_32m3cp = PPO('MlpPolicy', mon_env, verbose=0)
    eval_save_path = os.path.join('saved_models', 'PPO', layout, 'eval', 'pair1', 'partner', 'checkpoint_' + str(keeping_evals['PPO_1']['cp5']))
    evals = np.array([])
    total_timesteps = 1500000
    num_episodes = int(total_timesteps/400)
    for ep in range(num_episodes):
        mon_env.reset()
        FCP_32m3cp.learn(total_timesteps=400)
        if ep%37==0:
            FCP_32m3cp_save_path = os.path.join('saved_models', 'FCP', layout, FCPtype, 'checkpoint_' + str(int((ep/37)+1)))
            FCP_32m3cp.save(FCP_32m3cp_save_path)
            mean_reward = run_env(FCP_32m3cp_save_path, eval_save_path, layout, num_episodes=1)
            evals = np.append(evals, mean_reward)
            print(f'ep{ep}: {mean_reward}')
            mon_env = add_PPO_partners(layout, keeping_32, FCPtype)
        print(f'completed episode {ep+1} of {num_episodes} for {FCPtype}')

    x_plots_32m3cp = evals
    y_plots_32m3cp = np.arange(0, 1500400, 37*400)
    plots['FCP_32m3cp'] = (x_plots_32m3cp, y_plots_32m3cp)

    FCPtype = '16m6cp'
    mon_env = reset_env(layout)
    mon_env = add_PPO_partners(layout, keeping_16, FCPtype)
    FCP_16m6cp = PPO('MlpPolicy', mon_env, verbose=0)
    eval_save_path = os.path.join('saved_models', 'PPO', layout, 'eval', 'pair1', 'partner', 'checkpoint_' + str(keeping_evals['PPO_1']['cp5']))
    evals = np.array([])
    total_timesteps = 1500000
    num_episodes = int(total_timesteps/400)
    for ep in range(num_episodes):
        mon_env.reset()
        FCP_16m6cp.learn(total_timesteps=400)
        if ep%37==0:
            FCP_16m6cp_save_path = os.path.join('saved_models', 'FCP', layout, FCPtype, 'checkpoint_' + str(int((ep/37)+1)))
            FCP_16m6cp.save(FCP_16m6cp_save_path)
            mean_reward = run_env(FCP_16m6cp_save_path, eval_save_path, layout, num_episodes=1)
            evals = np.append(evals, mean_reward)
            print(f'ep{ep}: {mean_reward}')
            mon_env = add_PPO_partners(layout, keeping_16, FCPtype)
        print(f'completed episode {ep+1} of {num_episodes} for {FCPtype}')

    x_plots_16m6cp = evals
    y_plots_16m6cp = np.arange(0, 1500400, 37*400)
    plots['FCP_16m6cp'] = (x_plots_16m6cp, y_plots_16m6cp)

    results_32m3cp = {}
    FCP_path = os.path.join('saved_models', 'FCP', layout, '32m3cp', 'checkpoint_102')

    for cp in range(1, 6):
        results_32m3cp['cp' + str(cp)] = []
        for partner_num in range(1, 11):
            try:
                cp_num = keeping_evals['PPO_' + str(partner_num)]['cp' + str(cp)]
            except:
                cp_num = 1
            partner_path = os.path.join('saved_models', 'PPO', layout, 'eval', 'pair' + str(partner_num), 'partner', 'checkpoint_' + str(cp_num))
            results_32m3cp['cp' + str(cp)].append(run_env(FCP_path, partner_path, layout, num_episodes=10))

    results_16m6cp = {}
    FCP_path = os.path.join('saved_models', 'FCP', layout, '16m6cp', 'checkpoint_102')

    for cp in range(1, 6):
        results_16m6cp['cp' + str(cp)] = []
        for partner_num in range(1, 11):
            try:
                cp_num = keeping_evals['PPO_' + str(partner_num)]['cp' + str(cp)]
            except:
                cp_num = 1
            partner_path = os.path.join('saved_models', 'PPO', layout, 'eval', 'pair' + str(partner_num), 'partner', 'checkpoint_' + str(cp_num))
            results_16m6cp['cp' + str(cp)].append(run_env(FCP_path, partner_path, layout, num_episodes=10))

    means_32m3cp = {key: np.mean(results_32m3cp[key]) for key in results_32m3cp}
    means_16m6cp = {key: np.mean(results_16m6cp[key]) for key in results_16m6cp}

    return results_32m3cp, means_32m3cp, results_16m6cp, means_16m6cp, plots

# Training and evaluation

In [45]:
keeping = {}
for ppo in range(1, 11):
        partner_cp_array = evaluate_eval_partners(ppo, 'random1')
        keeping.update(find_checkpoints(ppo, partner_cp_array, 5))
keeping_evals_random1 = keeping

keeping = {}
for ppo in range(1, 11):
        partner_cp_array = evaluate_eval_partners(ppo, 'random0')
        keeping.update(find_checkpoints(ppo, partner_cp_array, 5))
keeping_evals_random0 = keeping

results_unident_s = {}
ego_path = os.path.join('saved_models', 'PPO', 'unident_s', '16m6cp', 'pair1', 'ego', 'checkpoint_100')

for cp in range(1, 6):
        results_unident_s['cp' + str(cp)] = []
        for partner_num in range(1, 11):
                partner_path = os.path.join('saved_models', 'PPO', 'unident_s', 'eval', 'pair' + str(partner_num), 'partner', 'checkpoint_' + str(cp))
                results_unident_s['cp' + str(cp)].append(run_env(ego_path, partner_path, 'unident_s', num_episodes=10))

results_random1 = {}
ego_path = os.path.join('saved_models', 'PPO', 'random1', '16m6cp', 'pair1', 'ego', 'checkpoint_100')

for cp in range(1, 6):
        results_random1['cp' + str(cp)] = []
        for partner_num in range(1, 11):
                try:
                        cp_num = keeping_evals_random1['PPO_' + str(partner_num)]['cp' + str(cp)]
                except:
                        cp_num = 1
                partner_path = os.path.join('saved_models', 'PPO', 'random1', 'eval', 'pair' + str(partner_num), 'partner', 'checkpoint_' + str(cp_num))
                results_random1['cp' + str(cp)].append(run_env(ego_path, partner_path, 'random1', num_episodes=10))

results_random0 = {}
ego_path = os.path.join('saved_models', 'PPO', 'random0', '16m6cp', 'pair1', 'ego', 'checkpoint_100')

for cp in range(1, 6):
        results_random0['cp' + str(cp)] = []
        for partner_num in range(1, 11):
                try:
                        cp_num = keeping_evals_random0['PPO_' + str(partner_num)]['cp' + str(cp)]
                except:
                        cp_num = 1
                partner_path = os.path.join('saved_models', 'PPO', 'random0', 'eval', 'pair' + str(partner_num), 'partner', 'checkpoint_' + str(cp_num))
                results_random0['cp' + str(cp)].append(run_env(ego_path, partner_path, 'random0', num_episodes=10))

results_simple = {}
ego_path = os.path.join('saved_models', 'PPO', 'simple', '16m6cp', 'pair1', 'ego', 'checkpoint_100')

for cp in range(1, 6):
        results_simple['cp' + str(cp)] = []
        for partner_num in range(1, 11):
                try:
                        cp_num = keeping_evals_simple['PPO_' + str(partner_num)]['cp' + str(cp)]
                except:
                        cp_num = 1
                partner_path = os.path.join('saved_models', 'PPO', 'simple', 'eval', 'pair' + str(partner_num), 'partner', 'checkpoint_' + str(cp_num))
                results_simple['cp' + str(cp)].append(run_env(ego_path, partner_path, 'simple', num_episodes=10))

Loaded MediumLevelPlanner from c:\users\bpcur\pantheonrl\overcookedgym\human_aware_rl\overcooked_ai\overcooked_ai_py\data\planners\random1_am.pkl
Team 1, evaluated checkpoint 1. mean reward: 4.7
Loaded MediumLevelPlanner from c:\users\bpcur\pantheonrl\overcookedgym\human_aware_rl\overcooked_ai\overcooked_ai_py\data\planners\random1_am.pkl
Team 1, evaluated checkpoint 2. mean reward: 7.2
Loaded MediumLevelPlanner from c:\users\bpcur\pantheonrl\overcookedgym\human_aware_rl\overcooked_ai\overcooked_ai_py\data\planners\random1_am.pkl
Team 1, evaluated checkpoint 3. mean reward: 10.2
Loaded MediumLevelPlanner from c:\users\bpcur\pantheonrl\overcookedgym\human_aware_rl\overcooked_ai\overcooked_ai_py\data\planners\random1_am.pkl
Team 1, evaluated checkpoint 4. mean reward: 12.2
Loaded MediumLevelPlanner from c:\users\bpcur\pantheonrl\overcookedgym\human_aware_rl\overcooked_ai\overcooked_ai_py\data\planners\random1_am.pkl
Team 1, evaluated checkpoint 5. mean reward: 16.1
Loaded MediumLevelPlan

In [50]:
means_simple = {key: np.mean(results_simple[key]) for key in results_simple}
print(means_simple)

{'cp1': 8.900000000000002, 'cp2': 57.48, 'cp3': 85.42999999999999, 'cp4': 120.8, 'cp5': 136.41000000000003}


In [51]:
means_random0 = {key: np.mean(results_random0[key]) for key in results_random0}
print(means_random0)

{'cp1': 5.27, 'cp2': 8.41, 'cp3': 19.05, 'cp4': 34.690000000000005, 'cp5': 46.07000000000001}


In [52]:
means_random1 = {key: np.mean(results_random1[key]) for key in results_random1}
print(means_random1)

{'cp1': 50.33, 'cp2': 101.51, 'cp3': 83.46000000000001, 'cp4': 81.29, 'cp5': 81.76000000000002}


In [54]:
means_unident_s = {key: np.mean(results_unident_s[key]) for key in results_unident_s}
print(means_unident_s)

{'cp1': 31.27, 'cp2': 114.46, 'cp3': 233.93999999999997, 'cp4': 332.03999999999996, 'cp5': 418.35}


# Results

## Asymmetric Advantages Results

In [23]:
results_32m3cp_unident_s = {'cp1': [35.8, 36.2, 43.4, 39.0, 29.4, 25.9, 31.1, 25.9, 31.9, 36.5],
 'cp2': [132.2, 96.4, 83.6, 107.8, 93.1, 126.9, 130.3, 94.9, 113.1, 118.1],
 'cp3': [241.9, 190.8, 235.6, 181.0, 216.6, 203.8, 194.9, 231.4, 200.3, 200.3],
 'cp4': [293.8, 325.2, 314.9, 266.1, 322.9, 310.3, 298.8, 296.8, 297.5, 268.5],
 'cp5': [403.3, 395.7, 380.3, 384.4, 380.9, 392.3, 372.0, 377.1, 387.2, 355.9]}

In [24]:
results_16m6cp_unident_s = {'cp1': [28.1, 20.4, 25.4, 13.7, 44.5, 33.2, 18.2, 34.1, 25.9, 32.2],
 'cp2': [119.4, 90.9, 109.6, 122.3, 77.0, 109.0, 112.7, 106.7, 104.5, 104.0],
 'cp3': [227.1, 228.8, 223.3, 207.9, 227.5, 227.5, 182.1, 197.4, 186.1, 174.0],
 'cp4': [286.4, 341.0, 315.6, 296.4, 307.3, 281.1, 295.1, 291.7, 297.5, 255.6],
 'cp5': [402.6, 393.7, 389.0, 379.8, 396.7, 392.1, 379.8, 359.1, 391.8, 348.4]}

In [25]:
means_16m6cp_unident_s = {key: np.mean(results_16m6cp_unident_s[key]) for key in results_16m6cp_unident_s}
means_32m3cp_unident_s = {key: np.mean(results_32m3cp_unident_s[key]) for key in results_32m3cp_unident_s}

In [26]:
means_16m6cp_unident_s

{'cp1': 27.57,
 'cp2': 105.60999999999999,
 'cp3': 208.17,
 'cp4': 296.77000000000004,
 'cp5': 383.30000000000007}

In [27]:
means_32m3cp_unident_s

{'cp1': 33.51,
 'cp2': 109.64000000000001,
 'cp3': 209.66,
 'cp4': 299.48,
 'cp5': 382.90999999999997}

In [56]:
SP_unident_s = {'cp1': [34.7, 36.9, 25.6, 35.4, 21.1, 25.3, 35.8, 20.5, 35.4, 42.0],
 'cp2': [139.1, 85.6, 100.5, 145.4, 93.2, 134.7, 129.8, 98.6, 118.1, 99.6],
 'cp3': [240.0, 256.0, 269.0, 184.1, 241.1, 253.8, 206.3, 268.4, 213.7, 207.0],
 'cp4': [334.4, 362.1, 340.8, 333.0, 368.6, 325.6, 336.9, 338.5, 323.6, 256.9],
 'cp5': [443.8, 429.7, 434.4, 431.3, 423.9, 445.7, 398.4, 402.2, 426.5, 347.6]}

## Cramped Room Results

In [28]:
results_32m3cp_simple = {'cp1': [278.5, 279.6, 291.0, 287.5, 286.6, 285.5, 288.8, 276.3, 266.9, 290.3],
 'cp2': [269.0, 271.3, 302.5, 255.8, 258.0, 267.4, 273.7, 271.2, 271.8, 292.2],
 'cp3': [271.2, 258.7, 307.7, 245.8, 263.7, 299.2, 289.4, 267.4, 254.3, 324.5],
 'cp4': [241.1, 281.6, 352.7, 261.1, 249.7, 371.0, 305.3, 264.1, 213.2, 370.4],
 'cp5': [201.3, 209.9, 364.7, 247.0, 183.3, 383.9, 363.8, 286.4, 140.0, 392.6]}

In [29]:
results_16m6cp_simple = {'cp1': [293.6, 287.2, 289.1, 308.4, 298.0, 296.2, 287.4, 282.0, 296.1, 297.4],
 'cp2': [255.3, 272.4, 293.2, 233.7, 283.2, 274.7, 257.0, 283.6, 267.9, 287.1],
 'cp3': [260.7, 296.5, 286.3, 221.5, 267.4, 316.3, 282.8, 278.3, 237.1, 258.8],
 'cp4': [257.2, 296.3, 301.5, 202.7, 252.3, 335.1, 269.6, 290.7, 219.4, 364.7],
 'cp5': [254.1, 278.5, 293.2, 193.5, 196.8, 319.2, 284.2, 299.5, 132.5, 414.7]}

In [30]:
means_16m6cp_simple = {'cp1': 293.53999999999996,
 'cp2': 270.81,
 'cp3': 270.57000000000005,
 'cp4': 278.95,
 'cp5': 266.62}

In [31]:
means_32m3cp_simple = {'cp1': 283.1,
 'cp2': 273.28999999999996,
 'cp3': 278.19,
 'cp4': 291.02000000000004,
 'cp5': 277.29}

In [55]:
SP_simple = {'cp1': [9.6, 5.4, 8.2, 6.3, 7.9, 6.8, 10.0, 11.0, 10.4, 13.4],
 'cp2': [17.1, 17.6, 107.3, 19.8, 17.1, 120.6, 68.8, 43.2, 0.9, 162.4],
 'cp3': [7.2, 36.0, 105.5, 10.7, 19.3, 169.5, 170.0, 71.8, 0.0, 264.3],
 'cp4': [9.3, 30.3, 160.1, 4.2, 1.8, 287.2, 242.7, 125.5, 0.0, 346.9],
 'cp5': [0.9, 24.7, 264.1, 1.2, 0.6, 326.0, 300.1, 70.2, 0.0, 376.3]}

## Forced Coordination Results

In [32]:
results_32m3cp_random0 = {'cp1': [14.5, 13.2, 11.8, 12.9, 14.5, 11.3, 18.8, 8.7, 13.0, 11.2],
 'cp2': [188.8, 118.7, 97.9, 136.4, 156.7, 78.0, 113.2, 167.6, 59.1, 129.4],
 'cp3': [264.2, 252.4, 187.6, 226.6, 160.2, 135.4, 259.8, 191.7, 84.1, 235.9],
 'cp4': [338.2, 282.1, 292.3, 278.8, 282.8, 144.3, 318.9, 258.9, 88.5, 297.5],
 'cp5': [362.2, 325.8, 360.7, 318.0, 291.6, 161.6, 300.1, 314.7, 147.9, 343.5]}

In [33]:
results_16m6cp_random0 = {'cp1': [16.1, 25.7, 20.8, 19.5, 11.2, 17.3, 11.1, 8.0, 8.1, 12.1],
 'cp2': [178.4, 141.4, 148.5, 127.6, 161.3, 75.0, 174.3, 161.6, 70.0, 108.6],
 'cp3': [287.7, 279.9, 269.6, 178.2, 197.3, 153.6, 276.7, 244.9, 75.1, 257.8],
 'cp4': [300.8, 250.1, 314.8, 201.1, 281.6, 179.9, 307.3, 310.2, 78.2, 304.8],
 'cp5': [329.1, 236.2, 381.5, 312.4, 227.7, 160.4, 340.5, 314.2, 109.7, 324.1]}

In [34]:
means_32m3cp_random0 = {'cp1': 12.99,
 'cp2': 124.58,
 'cp3': 199.79000000000002,
 'cp4': 258.23,
 'cp5': 292.61}

In [35]:
means_16m6cp_random0 = {'cp1': 14.989999999999998,
 'cp2': 134.67000000000002,
 'cp3': 222.07999999999998,
 'cp4': 252.88000000000002,
 'cp5': 273.58}

In [57]:
results_random0 = {'cp1': [4.8, 4.8, 5.7, 6.4, 8.5, 2.7, 6.0, 5.7, 4.8, 3.3],
 'cp2': [0.0, 1.2, 0.0, 0.3, 2.7, 14.4, 0.0, 0.0, 64.0, 1.5],
 'cp3': [0.0, 0.0, 0.0, 0.3, 3.3, 63.9, 0.0, 0.3, 122.7, 0.0],
 'cp4': [0.3, 0.6, 0.0, 0.0, 2.1, 158.8, 0.0, 0.3, 184.8, 0.0],
 'cp5': [0.0, 0.0, 0.0, 0.0, 0.0, 211.3, 0.0, 0.0, 249.4, 0.0]}

## Coordination Ring Results

In [36]:
results_32m3cp_random1 = {'cp1': [161.7, 183.0, 182.4, 184.2, 183.7, 174.8, 187.2, 176.2, 180.0, 190.6],
 'cp2': [271.6, 270.6, 233.9, 219.6, 200.2, 166.5, 215.1, 207.0, 213.4, 222.6],
 'cp3': [274.5, 291.1, 222.7, 209.3, 169.3, 166.3, 196.2, 243.6, 194.2, 209.3],
 'cp4': [246.2, 283.1, 231.6, 168.2, 134.6, 194.0, 254.1, 253.8, 202.6, 243.6],
 'cp5': [278.5, 266.0, 204.0, 197.2, 182.5, 219.1, 264.2, 215.4, 161.0, 224.7]}

In [37]:
results_16m6cp_random1 = {'cp1': [190.3, 187.8, 188.1, 171.3, 189.9, 191.6, 188.0, 188.2, 198.8, 182.1],
 'cp2': [264.7, 291.9, 244.0, 258.5, 204.5, 170.6, 237.0, 190.9, 160.2, 223.7],
 'cp3': [280.5, 311.7, 247.8, 201.5, 167.2, 172.7, 225.2, 219.0, 174.4, 221.3],
 'cp4': [291.7, 315.6, 196.7, 157.1, 144.3, 192.9, 271.9, 231.5, 136.7, 198.1],
 'cp5': [285.8, 258.5, 204.3, 227.9, 144.4, 122.9, 258.3, 179.3, 127.1, 197.9]}

In [38]:
means_32m3cp = {'cp1': 180.37999999999997,
 'cp2': 222.05,
 'cp3': 217.65,
 'cp4': 221.17999999999998,
 'cp5': 221.26}

In [39]:
means_16m6cp = {'cp1': 187.60999999999999,
 'cp2': 224.6,
 'cp3': 222.13000000000002,
 'cp4': 213.65,
 'cp5': 200.64000000000001}

In [58]:
results_random1 = {'cp1': [49.9, 65.7, 44.3, 42.1, 53.5, 37.3, 44.6, 68.9, 70.0, 27.0],
 'cp2': [242.6, 166.4, 125.4, 53.4, 20.7, 51.1, 82.2, 59.2, 115.5, 98.6],
 'cp3': [266.1, 106.6, 99.1, 49.6, 7.8, 37.5, 17.1, 63.8, 120.2, 66.8],
 'cp4': [272.0, 85.1, 89.8, 55.4, 8.8, 32.5, 28.9, 38.1, 115.1, 87.2],
 'cp5': [272.6, 166.8, 47.7, 67.3, 7.6, 22.5, 36.6, 23.4, 66.9, 106.2]}